In [17]:
# Usual Libraries USED FROM THE OTHER NOTEBOOK WE ADD MORE OR DELETE WHEN NECESSESARRY 
import pandas as pd
import numpy as np
import distutils
import ast

import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tqdm import tqdm

In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# Define the context features
context_features = {
    'video_id': tf.io.FixedLenFeature([], tf.string),
    'start_time_seconds': tf.io.FixedLenFeature([], tf.float32),
    'end_time_seconds': tf.io.FixedLenFeature([], tf.float32),
    'labels': tf.io.VarLenFeature(tf.int64)
}

# Define the sequence features
sequence_features = {
    'audio_embedding': tf.io.VarLenFeature(tf.string)
}

def _parse_function(example_proto):
    # Parse the example into context and sequence features
    context_data, sequence_data = tf.io.parse_single_sequence_example(
        example_proto,
        context_features=context_features,
        sequence_features=sequence_features
    )
    
    # Decode the audio embeddings from the byte strings
    audio_embeddings = tf.io.decode_raw(sequence_data['audio_embedding'].values, tf.uint8)
    audio_embeddings = tf.reshape(audio_embeddings, [-1, 128])  # Reshape to [time_steps, embedding_size]
    
    # Normalize the embeddings to the range [0, 1]
    audio_embeddings = tf.cast(audio_embeddings, tf.float32) / 255.0
    
    # Flatten the embedding to a 1D array
    flattened_embedding = tf.reshape(audio_embeddings, [-1])
    
    # Convert sparse labels to dense
    labels = tf.sparse.to_dense(context_data['labels'])
    
    return flattened_embedding, labels

def parse_tfrecords_to_dataframe(tfrecord_files):
    # Initialize an empty list to store embeddings and labels
    embeddings_list = []
    labels_list = []

    # Iterate through each TFRecord file in the folder
    for tfrecord_file in tqdm(tfrecord_files, desc="Processing TFRecord files"):
        raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
        parsed_dataset = raw_dataset.map(_parse_function)

        # Iterate over the parsed dataset and collect embeddings and labels
        for audio_embedding, labels in parsed_dataset:
            # Convert to numpy arrays
            audio_embedding_np = audio_embedding.numpy()
            labels_np = labels.numpy()

            # Flatten the label array if necessary
            labels_np = labels_np.flatten() if len(labels_np.shape) > 0 else labels_np

            # Store the embedding and label
            embeddings_list.append(audio_embedding_np)
            labels_list.append(labels_np)

    # Create a DataFrame from the collected data
    df = pd.DataFrame({
        'embedding': embeddings_list,
        'labels': labels_list
    })

    return df

# Specify the folder containing TFRecord files
bal_tfrecord_folder = '.github\Data\\bal_train'
bal_tfrecord_files = [os.path.join(bal_tfrecord_folder, f) for f in os.listdir(bal_tfrecord_folder) if f.endswith('.tfrecord')]

eval_tfrecord_folder = '.github\Data\\eval'
eval_tfrecord_files = [os.path.join(eval_tfrecord_folder, f) for f in os.listdir(eval_tfrecord_folder) if f.endswith('.tfrecord')]
eval_df = parse_tfrecords_to_dataframe(eval_tfrecord_files)
# Parse the TFRecords into a DataFrame
bal_df = parse_tfrecords_to_dataframe(bal_tfrecord_files)




Processing TFRecord files: 100%|██████████| 1444/1444 [02:50<00:00,  8.45it/s]


In [22]:
import os

# Ensure the 'data' directory exists
output_folder = r'.github\Data'
os.makedirs(output_folder, exist_ok=True)

# Paths for saving the CSV files
eval_csv_path = os.path.join(output_folder, 'eval_df.csv')
bal_csv_path = os.path.join(output_folder, 'bal_df.csv')

# Save the DataFrames to CSV
eval_df.to_csv(eval_csv_path, index=False)
bal_df.to_csv(bal_csv_path, index=False)




Reshape them  embeddings

In [26]:
import numpy as np

# Parameters (adjust as necessary)
embedding_length = 128  # This should match the original dimension of each heatmap slice
sequence_length = 100  # Adjust to match the desired number of time steps per sequence

# Convert the embedding column to a numpy array and reshape each embedding
def reshape_embeddings(df, embedding_length, sequence_length):
    reshaped_embeddings = []
    for embedding in df['embedding']:
        # Ensure embedding is a numpy array
        embedding = np.array(embedding)
        
        # Reshape the embedding into (sequence_length, embedding_length)
        # If necessary, truncate or pad to the sequence_length
        if len(embedding) < sequence_length * embedding_length:
            # Pad with zeros if too short
            padding_length = sequence_length * embedding_length - len(embedding)
            embedding = np.pad(embedding, (0, padding_length), mode='constant')
        elif len(embedding) > sequence_length * embedding_length:
            # Truncate if too long
            embedding = embedding[:sequence_length * embedding_length]
        
        # Reshape to (sequence_length, embedding_length)
        embedding = embedding.reshape((sequence_length, embedding_length))
        reshaped_embeddings.append(embedding)

    # Convert to numpy array for model input
    reshaped_embeddings = np.array(reshaped_embeddings)
    return reshaped_embeddings

# Apply reshaping to embeddings
reshaped_bal_embeddings = reshape_embeddings(bal_df, embedding_length, sequence_length)
#reshaped_eval_embeddings = reshape_embeddings(eval_df, embedding_length, sequence_length)
# Output the reshaped array's shape to verify
print("Reshaped Embeddings Shape:", reshaped_bal_embeddings.shape)
#print("Reshaped Embeddings Shape:", reshaped_eval_embeddings.shape)

Reshaped Embeddings Shape: (7971, 100, 128)
Reshaped Embeddings Shape: (7330, 100, 128)


In [30]:
import numpy as np
import librosa

# Process a single embedding to extract chroma
def extract_chroma(embedding):
    # Flatten the (100, 128) embedding to 1D
    embedding = embedding.flatten()  

    # Normalize the embedding to the range [-1, 1] (since librosa expects audio data in this range)
    embedding = embedding / np.max(np.abs(embedding))

    # Ensure the embedding is of the correct floating-point type
    embedding = np.float32(embedding)

    # Set the sampling rate
    sr = 22050  # Adjust as needed, typically for audio

    # Extract chroma
    chroma = librosa.feature.chroma_cqt(y=embedding, sr=sr)

    return chroma

# Apply chroma extraction to the balanced dataset (assuming reshaped_bal_embeddings is a list of embeddings)
chroma_features_bal = [extract_chroma(embedding) for embedding in reshaped_bal_embeddings]

# Check the shape of the first chroma feature
print(chroma_features_bal[0].shape)


TypeError: bad operand type for abs(): 'ellipsis'

In [ ]:
chroma_features_bal.shape

NameError: name 'chroma_features_bal' is not defined

Splitting the data and training 

In [ ]:
# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(reshaped_embeddings, df['embedding '], test_size=0.3, random_state=42)
 X_test, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Prepare lengths for CTC loss
input_lengths_train = np.full((len(X_train),), sequence_length)
label_lengths_train = np.array([len(emebedding ) for embedding  in y_train])

In [ ]:
# Train the model
model.fit(
    [X_train, y_train, input_lengths_train, label_lengths_train],
    np.zeros(len(y_train)),  # Dummy target required by Keras for Lambda layer output
    validation_data=(
        [X_val, y_val, np.full((len(X_val),), sequence_length), np.array([len(labels) for labels in y_val])],
        np.zeros(len(y_val))
    ),
    batch_size=32,
    epochs=20,
    callbacks=[tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')]
)
